In [26]:
push!(LOAD_PATH,".")

7-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/skh/MultiAgentSensing"
 "."
 "."
 "."

In [27]:
using Revise
using POMDPs

In [28]:
using MDMA

In [29]:
sensor = ViewConeSensor(pi/2, 3);
targets = Vector{Target}(undef, 0);
push!(targets, Target(5,2,0));
push!(targets, Target(5,3,0));
push!(targets, Target(4,3,0));

grid = Grid(10,10);
horizon = 30
move_dist = 3
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, targets, move_dist);

In [30]:
policy = MDMA.solve_single_robot(model, UAVState(1,1,:N))


[Iteration 1   ] residual:        120 | iteration runtime:    140.567 ms, (     0.141 s total)
[Iteration 2   ] residual:        120 | iteration runtime:    127.613 ms, (     0.268 s total)
[Iteration 3   ] residual:        120 | iteration runtime:    134.228 ms, (     0.402 s total)
[Iteration 4   ] residual:        120 | iteration runtime:    134.291 ms, (     0.537 s total)
[Iteration 5   ] residual:        120 | iteration runtime:    128.516 ms, (     0.665 s total)
UAVState(3, 1, :NE, false)nothing

ValueIterationPolicy:
 MDPState(UAVState(1, 1, :E, false), 0, nothing) -> UAVState(3, 3, :SE, false)
 MDPState(UAVState(1, 2, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(1, 3, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(1, 4, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(1, 5, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(1, 6, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(1, 7, :E, false), 0, nothing) -> UAVState(3, 5, :SE, false)
 MDPState(UAVState(1, 8, :E, false), 0, nothing) -> UAVState(2, 6, :SE, false)
 MDPState(UAVState(1, 9, :E, false), 0, nothing) -> UAVState(3, 7, :SE, false)
 MDPState(UAVState(1, 10, :E, false), 0, nothing) -> UAVState(2, 8, :SE, false)
 MDPState(UAVState(2, 1, :E, false), 0, nothing) -> UAVState(3, 3, :SE, false)
 MDPState(UAVState(2, 2, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(2, 3, :E, 

In [31]:
function rew_check(model, state, policy)
    prev = MDPState(state)
    next = action(policy, prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end
function check(model, prev, next)
    prev = MDPState(prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end


check (generic function with 1 method)

In [32]:
test_state = UAVState(3,3,:S)
rew_check(model, test_state, policy)

UAVState(3, 3, :S, false) 0
UAVState(5, 4, :SW, false) 15


In [33]:
check(model, test_state, UAVState(3,3,:E))

UAVState(3, 3, :S, false) 0
UAVState(3, 3, :E, false) 15


In [34]:
nbors = MDMA.neighbors(model.grid, test_state, move_dist)

50-element Vector{UAVState}:
 UAVState(1, 1, :SE, false)
 UAVState(1, 1, :SW, false)
 UAVState(1, 2, :SE, false)
 UAVState(1, 2, :SW, false)
 UAVState(1, 3, :SE, false)
 UAVState(1, 3, :SW, false)
 UAVState(1, 4, :SE, false)
 UAVState(1, 4, :SW, false)
 UAVState(1, 5, :SE, false)
 UAVState(1, 5, :SW, false)
 UAVState(2, 1, :SE, false)
 UAVState(2, 1, :SW, false)
 UAVState(2, 2, :SE, false)
 ⋮
 UAVState(4, 5, :SE, false)
 UAVState(4, 5, :SW, false)
 UAVState(5, 1, :SE, false)
 UAVState(5, 1, :SW, false)
 UAVState(5, 2, :SE, false)
 UAVState(5, 2, :SW, false)
 UAVState(5, 3, :SE, false)
 UAVState(5, 3, :SW, false)
 UAVState(5, 4, :SE, false)
 UAVState(5, 4, :SW, false)
 UAVState(5, 5, :SE, false)
 UAVState(5, 5, :SW, false)

In [23]:
UAVState(3,3,:E) in nbors

false

In [24]:
test_state in nbors

false

In [25]:
nbors

50-element Vector{UAVState}:
 UAVState(1, 1, :SE, false)
 UAVState(1, 1, :SW, false)
 UAVState(1, 2, :SE, false)
 UAVState(1, 2, :SW, false)
 UAVState(1, 3, :SE, false)
 UAVState(1, 3, :SW, false)
 UAVState(1, 4, :SE, false)
 UAVState(1, 4, :SW, false)
 UAVState(1, 5, :SE, false)
 UAVState(1, 5, :SW, false)
 UAVState(2, 1, :SE, false)
 UAVState(2, 1, :SW, false)
 UAVState(2, 2, :SE, false)
 ⋮
 UAVState(4, 5, :SE, false)
 UAVState(4, 5, :SW, false)
 UAVState(5, 1, :SE, false)
 UAVState(5, 1, :SW, false)
 UAVState(5, 2, :SE, false)
 UAVState(5, 2, :SW, false)
 UAVState(5, 3, :SE, false)
 UAVState(5, 3, :SW, false)
 UAVState(5, 4, :SE, false)
 UAVState(5, 4, :SW, false)
 UAVState(5, 5, :SE, false)
 UAVState(5, 5, :SW, false)